In [1]:
import pandas as pd
import warnings
import re
import numpy as np
import lightgbm as lgb
from IPython.display import HTML
from sklearn.metrics import roc_auc_score
# 描画関連
pd.options.display.precision = 15
warnings.filterwarnings("ignore")
# COL数
pd.set_option("display.max_columns", None)
# ROW数
pd.set_option('display.max_rows', 1000)
# 幅
display(HTML("<style>.container { width:100% !important; }</style>"))

# 最後の出力
print('All components loads was done! Ready!')

All components loads was done! Ready!


In [2]:
df = pd.read_csv("./var/collect_data.csv")

In [3]:
df

,rank,race_time,tansho_odds,fukusho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob
0,4,"1'53""1",2.0,1.2-1.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,1,4816,A1,村松 修二,広島,広島29歳,51.9kg,F0,L0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98
1,6,NaN,4.8,2.2-3.2,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,2,4914,A2,吉田 裕平,愛知,愛知23歳,51.0kg,F0,L0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07
2,1,"1'49""2",4.6,1.8-2.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,3,4908,A1,上田 龍星,大阪,大阪24歳,51.0kg,F0,L0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56
3,2,"1'50""7",5.1,1.6-2.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,4,5028,B1,原田 才一郎,福岡,福岡23歳,52.5kg,F0,L0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78
4,5,NaN,5.7,2.3-3.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,5,4776,A2,竹田 和哉,香川,愛媛26歳,52.5kg,F0,L0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,"1'51""1",NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,2,3550,A1,都築 正治,愛知,愛知47歳,51.0kg,F0,L0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65
1232864,6,NaN,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,3,3773,A2,津留 浩一郎,長崎,長崎44歳,52.3kg,F0,L0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41
1232865,1,"1'49""6",NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,4,4382,B1,高山 智至,滋賀,京都34歳,53.9kg,F0,L0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62
1232866,4,"1'53""4",NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,5,3709,B1,田中 浩之,山口,山口44歳,51.2kg,F0,L0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22


In [4]:
# race_time sanitize

def race_time_sanitize(x):
    if not pd.isna(x):
        h, t = x.split("'")
        t = t.replace('"', ".")
        return int(h) * 60 + float(t)
    else:
        return None
df["race_time"] = df["race_time"].apply(race_time_sanitize)

df

,rank,race_time,tansho_odds,fukusho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob
0,4,113.099999999999994,2.0,1.2-1.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,1,4816,A1,村松 修二,広島,広島29歳,51.9kg,F0,L0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98
1,6,NaN,4.8,2.2-3.2,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,2,4914,A2,吉田 裕平,愛知,愛知23歳,51.0kg,F0,L0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07
2,1,109.200000000000003,4.6,1.8-2.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,3,4908,A1,上田 龍星,大阪,大阪24歳,51.0kg,F0,L0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56
3,2,110.700000000000003,5.1,1.6-2.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,4,5028,B1,原田 才一郎,福岡,福岡23歳,52.5kg,F0,L0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78
4,5,NaN,5.7,2.3-3.3,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,5,4776,A2,竹田 和哉,香川,愛媛26歳,52.5kg,F0,L0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,111.099999999999994,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,2,3550,A1,都築 正治,愛知,愛知47歳,51.0kg,F0,L0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65
1232864,6,NaN,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,3,3773,A2,津留 浩一郎,長崎,長崎44歳,52.3kg,F0,L0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41
1232865,1,109.599999999999994,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,4,4382,B1,高山 智至,滋賀,京都34歳,53.9kg,F0,L0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62
1232866,4,113.400000000000006,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,5,3709,B1,田中 浩之,山口,山口44歳,51.2kg,F0,L0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22


In [5]:
# split fukusho to min, max
def split_fukusho_to_min_max(x, index=0):
    if not pd.isna(x):
        try:
            min_, max_ = x.split("-")
            min_, max_ = [float(min_), float(max_)]
            return [min_, max_][index]
        except:
            return None
    else:
        return None

df["fukusho_min"] = df["fukusho_odds"].apply(lambda x:split_fukusho_to_min_max(x, 0))
df["fukusho_max"] = df["fukusho_odds"].apply(lambda x:split_fukusho_to_min_max(x, 1))

df.drop(["fukusho_odds"], axis=1, inplace=True)
df

,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,fukusho_min,fukusho_max
0,4,113.099999999999994,2.0,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,1,4816,A1,村松 修二,広島,広島29歳,51.9kg,F0,L0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98,1.2,1.3
1,6,NaN,4.8,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,2,4914,A2,吉田 裕平,愛知,愛知23歳,51.0kg,F0,L0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07,2.2,3.2
2,1,109.200000000000003,4.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,3,4908,A1,上田 龍星,大阪,大阪24歳,51.0kg,F0,L0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56,1.8,2.6
3,2,110.700000000000003,5.1,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,4,5028,B1,原田 才一郎,福岡,福岡23歳,52.5kg,F0,L0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78,1.6,2.3
4,5,NaN,5.7,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,5,4776,A2,竹田 和哉,香川,愛媛26歳,52.5kg,F0,L0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62,2.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,111.099999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,2,3550,A1,都築 正治,愛知,愛知47歳,51.0kg,F0,L0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65,NaN,NaN
1232864,6,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,3,3773,A2,津留 浩一郎,長崎,長崎44歳,52.3kg,F0,L0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41,NaN,NaN
1232865,1,109.599999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,4,4382,B1,高山 智至,滋賀,京都34歳,53.9kg,F0,L0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62,NaN,NaN
1232866,4,113.400000000000006,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,5,3709,B1,田中 浩之,山口,山口44歳,51.2kg,F0,L0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22,NaN,NaN


In [6]:
# age sanitize
def age_sanitize(x):
    try:
        return int(re.search("(\d{1,})", x).group(1))
    except:
        return None

df["age"] = df["age"].apply(age_sanitize)
df

,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,fukusho_min,fukusho_max
0,4,113.099999999999994,2.0,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,1,4816,A1,村松 修二,広島,29,51.9kg,F0,L0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98,1.2,1.3
1,6,NaN,4.8,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,2,4914,A2,吉田 裕平,愛知,23,51.0kg,F0,L0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07,2.2,3.2
2,1,109.200000000000003,4.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,3,4908,A1,上田 龍星,大阪,24,51.0kg,F0,L0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56,1.8,2.6
3,2,110.700000000000003,5.1,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,4,5028,B1,原田 才一郎,福岡,23,52.5kg,F0,L0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78,1.6,2.3
4,5,NaN,5.7,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,5,4776,A2,竹田 和哉,香川,26,52.5kg,F0,L0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62,2.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,111.099999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,2,3550,A1,都築 正治,愛知,47,51.0kg,F0,L0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65,NaN,NaN
1232864,6,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,3,3773,A2,津留 浩一郎,長崎,44,52.3kg,F0,L0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41,NaN,NaN
1232865,1,109.599999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,4,4382,B1,高山 智至,滋賀,34,53.9kg,F0,L0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62,NaN,NaN
1232866,4,113.400000000000006,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,5,3709,B1,田中 浩之,山口,44,51.2kg,F0,L0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22,NaN,NaN


In [7]:
# weight sanitize
def weight_sanitize(x):
    try:
        return re.search("(\d\d.\d)", x).group(1)
    except:
        return None
df["weight"] = df["weight"].apply(weight_sanitize)
df

,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,fukusho_min,fukusho_max
0,4,113.099999999999994,2.0,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,1,4816,A1,村松 修二,広島,29,51.9,F0,L0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98,1.2,1.3
1,6,NaN,4.8,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,2,4914,A2,吉田 裕平,愛知,23,51.0,F0,L0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07,2.2,3.2
2,1,109.200000000000003,4.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,3,4908,A1,上田 龍星,大阪,24,51.0,F0,L0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56,1.8,2.6
3,2,110.700000000000003,5.1,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,4,5028,B1,原田 才一郎,福岡,23,52.5,F0,L0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78,1.6,2.3
4,5,NaN,5.7,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,芦屋,5,4776,A2,竹田 和哉,香川,26,52.5,F0,L0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62,2.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,111.099999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,2,3550,A1,都築 正治,愛知,47,51.0,F0,L0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65,NaN,NaN
1232864,6,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,3,3773,A2,津留 浩一郎,長崎,44,52.3,F0,L0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41,NaN,NaN
1232865,1,109.599999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,4,4382,B1,高山 智至,滋賀,34,53.9,F0,L0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62,NaN,NaN
1232866,4,113.400000000000006,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,蒲郡,5,3709,B1,田中 浩之,山口,44,51.2,F0,L0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22,NaN,NaN


In [8]:
df.to_csv("boatrace.csv.gz", compression="gzip", index=None)

In [9]:
df["kaijo_name"].dtype.str

'|O'

In [10]:
# label encoding
for cat_col in ["class_name", "kaijo_name", "racer_name", "country", "fnum", "lnum"]:
    if df[cat_col].dtype.str == "|O":
        dic = {val:idx for idx, val in enumerate(df[cat_col].unique())}
        df[cat_col] = df[cat_col].apply(lambda x:dic[x])

df

,rank,race_time,tansho_odds,cup_name,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,fukusho_min,fukusho_max
0,4,113.099999999999994,2.0,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,0,1,4816,0,0,0,29,51.9,0,0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.62,72,29.410000000000000,50.98,1.2,1.3
1,6,NaN,4.8,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,0,2,4914,1,1,1,23,51.0,0,0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.86,60,37.500000000000000,66.07,2.2,3.2
2,1,109.200000000000003,4.6,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,0,3,4908,0,2,2,24,51.0,0,0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.55,52,15.560000000000000,35.56,1.8,2.6
3,2,110.700000000000003,5.1,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,0,4,5028,2,3,3,23,52.5,0,0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50,34,35.560000000000002,57.78,1.6,2.3
4,5,NaN,5.7,ルーキーシリーズ第１２戦 スカパー！・ＪＬＣカップ,0,5,4776,1,4,4,26,52.5,0,0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36,13,28.300000000000001,39.62,2.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232863,3,111.099999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,5,2,3550,0,939,1,47,51.0,0,0,0.17,6.20,44.700000000000003,63.640000000000001,7.75,62.750000000000000,84.310000000000002,14,37.890000000000001,58.95,58,35.140000000000001,48.65,NaN,NaN
1232864,6,NaN,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,5,3,3773,1,128,15,44,52.3,0,0,0.17,4.86,32.140000000000001,44.640000000000001,3.32,14.289999999999999,28.570000000000000,39,34.289999999999999,56.19,62,40.710000000000001,58.41,NaN,NaN
1232865,1,109.599999999999994,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,5,4,4382,2,1341,11,34,53.9,0,0,0.17,5.51,36.170000000000002,53.189999999999998,5.00,21.050000000000001,36.840000000000003,12,40.000000000000000,56.67,77,40.189999999999998,62.62,NaN,NaN
1232866,4,113.400000000000006,NaN,マンスリーＢＯＡＴＲＡＣＥ杯争奪戦,5,5,3709,2,274,14,44,51.2,0,0,0.21,4.08,17.440000000000001,31.399999999999999,3.86,0.000000000000000,28.570000000000000,48,42.000000000000000,57.00,23,29.270000000000000,51.22,NaN,NaN


In [11]:
# sanitize rank
def int_parse(x):
    try:
        return int(x)
    except:
        return None

df["rank"] = df["rank"].apply(int_parse)
df = df[pd.notnull(df["rank"])]

In [12]:
# sanitize tansho_odds, wight, mean_st, mortor_win_in_3_prob, boat_win_in_3_prob

def sanitize(x):
    try:
        return float(x)
    except:
        try:
            return int(x)
        except:
            return None

for col in ["tansho_odds", "weight", "mean_st", "mortor_win_in_3_prob", "boat_win_in_3_prob"]:
    df[col] = df[col].apply(sanitize)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1215009 entries, 0 to 1232867
Data columns (total 29 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   rank                   1215009 non-null  float64
 1   race_time              933334 non-null   float64
 2   tansho_odds            779748 non-null   float64
 3   cup_name               1215009 non-null  object 
 4   kaijo_name             1215009 non-null  int64  
 5   waku_name              1215009 non-null  int64  
 6   touroku_no             1215009 non-null  int64  
 7   class_name             1215009 non-null  int64  
 8   racer_name             1215009 non-null  int64  
 9   country                1215009 non-null  int64  
 10  age                    1215009 non-null  int64  
 11  weight                 1215009 non-null  float64
 12  fnum                   1215009 non-null  int64  
 13  lnum                   1215009 non-null  int64  
 14  mean_st           

In [13]:
df.shape

(1215009, 29)

In [14]:
df = df[pd.notnull(df["tansho_odds"])]

In [44]:
cup_names = df["cup_name"].unique()
size=len(cup_names)
# trn_cup val_cup = set(cup_names[:int(0.8*size)]), set(cup_names[int(0.8*size):])
import random
trn_cup = set(random.sample(cup_names.tolist(), int(0.8*size)))
val_cup = {c for c in cup_names if c not in trn_cup}

trn_y = df[df.cup_name.apply(lambda x:x in trn_cup)]["rank"].apply(lambda x:x<=2)
val_y = df[df.cup_name.apply(lambda x:x in val_cup)]["rank"].apply(lambda x:x<=2)

val_y

6          False
7           True
8          False
9          False
10         False
           ...  
1232845     True
1232846    False
1232847    False
1232848    False
1232849    False
Name: rank, Length: 153387, dtype: bool

In [69]:
trn_x = df[df.cup_name.apply(lambda x:x in trn_cup)].drop(["race_time", "rank", "cup_name"], axis=1)
val_x = df[df.cup_name.apply(lambda x:x in val_cup)].drop(["race_time", "rank", "cup_name"], axis=1)

trn_x

,tansho_odds,kaijo_name,waku_name,touroku_no,class_name,racer_name,country,age,weight,fnum,lnum,mean_st,zenkoku_win_in_1_prob,zenkoku_win_in_2_prob,zenkoku_win_in_3_prob,touchi_win_in_1_prob,touchi_win_in_2_prob,touchi_win_in_3_prob,mortor_win_in_1_prob,mortor_win_in_2_prob,mortor_win_in_3_prob,boat_win_in_1_prob,boat_win_in_2_prob,boat_win_in_3_prob,fukusho_min,fukusho_max
0,2.000000000000000,0,1,4816,0,0,0,29,51.899999999999999,0,0,0.14,7.53,60.770000000000003,75.379999999999995,6.23,50.000000000000000,59.090000000000003,61,23.079999999999998,34.619999999999997,72,29.410000000000000,50.979999999999997,1.2,1.3
1,4.800000000000000,0,2,4914,1,1,1,23,51.000000000000000,0,0,0.17,6.45,40.430000000000000,60.640000000000001,0.00,0.000000000000000,0.000000000000000,10,32.140000000000001,42.859999999999999,60,37.500000000000000,66.069999999999993,2.2,3.2
2,4.600000000000000,0,3,4908,0,2,2,24,51.000000000000000,0,0,0.13,5.79,37.250000000000000,51.960000000000001,5.59,34.380000000000003,59.380000000000003,41,42.420000000000002,54.549999999999997,52,15.560000000000000,35.560000000000002,1.8,2.6
3,5.100000000000000,0,4,5028,2,3,3,23,52.500000000000000,0,0,0.17,4.74,25.969999999999999,42.859999999999999,3.15,9.090000000000000,25.449999999999999,63,41.670000000000002,50.000000000000000,34,35.560000000000002,57.780000000000001,1.6,2.3
4,5.700000000000000,0,5,4776,1,4,4,26,52.500000000000000,0,0,0.16,5.98,36.670000000000002,52.219999999999999,6.00,30.000000000000000,66.670000000000002,8,24.000000000000000,36.000000000000000,13,28.300000000000001,39.619999999999997,2.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232809,4.200000000000000,13,2,4026,2,869,6,39,54.000000000000000,0,0,0.20,4.10,17.239999999999998,36.210000000000001,3.60,10.000000000000000,30.000000000000000,70,45.600000000000001,70.400000000000006,49,23.890000000000001,35.399999999999999,2.3,7.0
1232810,1.900000000000000,13,3,4108,0,48,14,37,51.000000000000000,1,0,0.16,6.85,58.119999999999997,72.650000000000006,7.41,55.170000000000002,68.969999999999999,46,37.170000000000002,56.640000000000001,43,21.510000000000002,36.560000000000002,1.0,1.7
1232811,0.000000000000000,13,4,4984,3,1141,5,20,49.500000000000000,0,0,NaN,1.78,0.000000000000000,4.000000000000000,0.00,0.000000000000000,0.000000000000000,48,30.190000000000001,43.399999999999999,59,26.609999999999999,40.369999999999997,0.0,0.0
1232812,12.699999999999999,13,5,4319,1,1408,2,32,53.200000000000003,2,0,0.17,6.52,47.369999999999997,68.420000000000002,6.27,51.350000000000001,59.460000000000001,26,39.090000000000003,55.450000000000003,40,34.859999999999999,55.049999999999997,2.3,7.0


In [70]:
trn_y

0          False
1          False
2           True
3           True
4          False
           ...  
1232809    False
1232810     True
1232811    False
1232812     True
1232813    False
Name: rank, Length: 626361, dtype: bool

In [98]:
param = {
            "objective": "binary",
            "metric": "auc",
            "max_depth": 15,
            "num_leaves": 15,
            "learning_rate": 0.001,
            #"bagging_fraction": 0.99,
            #"feature_fraction": 0.99,
            "lambda_l1": 0.3,
            "lambda_l2": 0.3,
            "bagging_seed": 777,
            "verbosity": -1,
            "seed": 777,
            # 'max_bin': 512
}

In [99]:
def train(trn_x, trn_y, val_x, val_y, category, param, verbose, early_stopping_rounds, n_estimators):

    print(f'now start to train')
    trn_data = lgb.Dataset(trn_x, label=trn_y, categorical_feature=category)
    val_data = lgb.Dataset(val_x, label=val_y, categorical_feature=category)
    num_round = n_estimators
    clf = lgb.train(param, trn_data, num_round, valid_sets=[
        trn_data, val_data], verbose_eval=verbose, early_stopping_rounds=early_stopping_rounds)
    #oof_predictions[xs_val_idx] = clf.predict(xs_val)
    eval_loss = roc_auc_score(val_y, clf.predict(val_x))
    print(f'end eval_loss={eval_loss}')
    return clf

In [100]:
category = [idx for idx, c in enumerate(trn_x.columns) if c in {"class_name", "kaijo_name", "racer_name", "country", "fnum", "lnum"}]
clf = train(trn_x, trn_y, val_x, val_y, category, param, 1, 5, 10000)

now start to train
[1]	training's auc: 0.782197	valid_1's auc: 0.783015
Training until validation scores don't improve for 5 rounds
[2]	training's auc: 0.782197	valid_1's auc: 0.783015
[3]	training's auc: 0.782197	valid_1's auc: 0.783015
[4]	training's auc: 0.782197	valid_1's auc: 0.783015
[5]	training's auc: 0.782197	valid_1's auc: 0.783015
[6]	training's auc: 0.782467	valid_1's auc: 0.783319
[7]	training's auc: 0.782587	valid_1's auc: 0.783508
[8]	training's auc: 0.782515	valid_1's auc: 0.783386
[9]	training's auc: 0.782595	valid_1's auc: 0.783522
[10]	training's auc: 0.782585	valid_1's auc: 0.783507
[11]	training's auc: 0.782595	valid_1's auc: 0.783522
[12]	training's auc: 0.783721	valid_1's auc: 0.784622
[13]	training's auc: 0.783721	valid_1's auc: 0.784622
[14]	training's auc: 0.783721	valid_1's auc: 0.784622
[15]	training's auc: 0.783721	valid_1's auc: 0.784619
[16]	training's auc: 0.783721	valid_1's auc: 0.784622
[17]	training's auc: 0.783731	valid_1's auc: 0.784634
[18]	trainin

In [101]:
imp = [{"column":c, "gain":g} for c,g in zip(trn_x.columns, clf.feature_importance(importance_type="gain"))]

pd.DataFrame(imp).sort_values(by=["gain"], ascending=False)

,column,gain
2,waku_name,2.670219446350098e+06
24,fukusho_min,1.051273138671875e+06
0,tansho_odds,8.539935496826172e+05
12,zenkoku_win_in_1_prob,1.468020614624023e+05
14,zenkoku_win_in_3_prob,0.000000000000000e+00
23,boat_win_in_3_prob,0.000000000000000e+00
22,boat_win_in_2_prob,0.000000000000000e+00
21,boat_win_in_1_prob,0.000000000000000e+00
20,mortor_win_in_3_prob,0.000000000000000e+00
19,mortor_win_in_2_prob,0.000000000000000e+00
